Credit: Đoàn Vĩnh Nhân - Toán Tin K68 - 20237376

### Phương pháp Gauss - Jordan
+ Input: Ma trận $A_{m\times n},B_{m\times q}$
+ Output: Ma trận $X_{n\times (q+k)}$, với $k$ là số hàng bằng 0 sau khi khử $AB$. Các phần tử trên cột q đặc trưng cho sự phụ thuộc của các phần tử còn lại của $X$ vào các phần tử tự do.

Ý tưởng thuật toán: khử ma trận tương tự gauss nhưng khác biệt ở cách chọn phần tử khử. Hạn chế sai số trong quá trình khử do chia cho số gần bằng 0

1. Xét ma trận mở rộng A|B, chọn phần tử khử a trong ma trận
   + Ưu tiên 1: $|a|$ là số nguyên dương nhỏ nhất
   + Ưu tiên 2: $|a| = \max\{|a_{ij}|\}$
   + Chọn phần tử khử thứ k có hàng và cột khác với k-1 phần tử trước đó

2. Cho $t$ chạy từ $1$ đến $m,\, t \ne p$ thực hiện: $$ L_t- \dfrac{a_{tq}}{a_{pq}}L_p\Rightarrow L_t \quad (*)$$


Thuật toán

1. Tạo 2 danh sách rowuse, coluse rỗng, dùng lưu hàng, cột của các phần tử khử đã chọn.
2. Tìm kiếm phần tử khử theo ưu tiên (1), (2) ở trong những hàng, cột khác các phần tử trong rowuse, coluse
3. Khử ma trận A|B theo công thức (*). Lưu vị trí cột, hàng của phần tử khử vào rowused,colused.
4. Nếu rowuse == row hoặc coluse == col hoặc pivot == 0(trường hợp có hàng bằng 0) đến bước 5, nếu không quay lại bước 2
5. Chuẩn hoá A|B : chia từng hàng cho pivot (nếu có) và sắp xếp về dạng bậc thang

6. Trả về kết quả nghiệm (vô nghiệm, nghiệm duy nhất, nghiệm tổng quát của trường hợp vô số nghiệm)
+ Chuyển các cột không có phần tử khử của $A$ ra sau $B$ (Các phần tử tự do)
+ Loại bỏ các hàng bằng 0
+ Trả về B (giờ đây B chính là X do các ma trận A giờ đây đã là ma trận đơn vị)

In [1]:
from sympy import Matrix, zeros, N
import numpy as np

#Hàm này được thêm vào để loại bỏ các giá trị nhỏ hơn 10⁻¹⁰, đảm bảo chúng được coi là 0.
def chop_small_values(mat):
    if isinstance(mat, Matrix):
        return mat.applyfunc(lambda x: 0 if abs(x) < 1e-10 else x)
    else:
        return 0 if abs(mat) < 1e-10 else mat

#Hàm tìm kiếm phần tử khử, trả về giá trị và vị trí của phần tử khử
def findpivot(AB,rowA,colA,rowused,colused):
    #Hàng và cột chưa dùng
    row = list(set(range(colA)) - set(rowused))
    col = list(set(range(colA)) - set(colused))
    
    #Ưu tiên 1 và ưu tiên 2
    firts_prioritize_element = [AB[i, j] for i in row for j in col
                                if float(AB[i, j]) == float(int(1))]
    if firts_prioritize_element:
        pivot = min(firts_prioritize_element, key=abs)
    else:
        second_prioritize_element = [AB[i, j] for i in row for j in col]
        pivot = max(second_prioritize_element, key=abs)
    positions = [[i, j] for i in row for j in col if AB[i, j] == pivot]
    return pivot, positions
#Hàm khử gauss_jordan
def gauss_jordan(A,B):

    A, B = Matrix(N(A)), Matrix(N(B))
    rowused = []
    colused = []
    rowA, colA = A.shape
    AB = chop_small_values(A.row_join(B))
    display(AB)
    standardization = []

    #Tìm pivot và khử theo công thức (1)
    while ((len(rowused)!=rowA) or (len(colused)!=colA)):
        pivot = findpivot(AB,rowA,colA,rowused,colused)[0]
        if (abs(pivot) < 1e-10): break
        pivotrow, pivotcol = findpivot(AB,rowA,colA,rowused,colused)[1][0]
        for i in [x for x in range(rowA) if x!=pivotrow]:
            AB[i,:] = AB[i,:] - AB[pivotrow,:]*AB[i,pivotcol]/pivot
        rowused.append(pivotrow)
        colused.append(pivotcol)
        standardization.append([pivotrow,pivotcol])
        print("Khử")
        AB = chop_small_values(AB)
        display(AB)

    #Chuẩn hoá, chia các hàng cho pivot
    print("Chuẩn hoá")
    for i in range(len(standardization)):
        pivotrow, pivotcol = standardization[i]
        AB[pivotrow,:] = AB[pivotrow,:]/AB[pivotrow,pivotcol]
    return chop_small_values(AB), colused
#Danh sách các vị trí khác 0 đầu tiên của từng hàng, dùng để sắp xếp AB theo cấu trúc bậc thang
def indexpivot(AB):
    rowAB, colAB = AB.shape
    index = []
    for row in range(rowAB):
        for col in range(colAB):
            if abs(AB[row, col]) > 1e-10: 
                index.append(col)
                break
            if col == colAB - 1:
                index.append(col + 1)
    return index
#Giải AX=B
def gauss_jordan_solve(A, B):
    #Ma trận đã khử AB
    A, B = Matrix(N(A)), Matrix(N(B))
    AB,colused = gauss_jordan(A, B)
    display(AB)
    #Kiểm tra trường hợp vô nghiệm
    for i in indexpivot(AB):
        if i >= A.shape[1] and i < AB.shape[1]:
            print("Hệ vô nghiệm")
            return 
    
    print(f"Số phần tử tự do là {len(list(set(range(A.shape[0]))-set(colused)))}: X{sorted([x+1 for x in list(set(range(A.shape[0]))-set(colused))])}.")

    return

In [2]:
A = Matrix(np.loadtxt("GJ_input_A.txt"))
print("A =")
display(A)
print("B =")
B = Matrix(np.loadtxt("GJ_input_B.txt"))
display(B)
print("A|B=")
gauss_jordan_solve(A,B)


A =


Matrix([
[ 130.75,    3.75,    5.25,    11.25,   18.25,     6.5,    12.0,     15.5,    2.75,     20.0],
[   -7.0,   156.0,     4.0,      8.0, 17.6667, -3.6667,    -3.0,      4.0,     9.0,  21.6667],
[    2.8,     4.2,    89.0,      5.4,    10.8,    12.6,    -7.8,      8.0,    14.8,     13.0],
[16.5714, 14.8571, 25.4286, 140.4286, 25.7143,  2.1429,  5.5714,  10.5714,  5.4286,   6.4286],
[ 6.1667,   -19.0,  9.8333,  12.1667,   114.0, -6.1667,    14.5,   3.6667,   -10.5, -14.1667],
[    0.0,     0.0,     0.0,      0.0,     0.0,     0.0,     0.0,      0.0,     0.0,      0.0],
[    0.0,     0.0,     0.0,      0.0,     0.0,     0.0,     0.0,      0.0,     0.0,      0.0],
[-7.5714, -1.4286,  5.2857,   9.8571,   -18.0,  7.1429, 96.8571, -10.1429, -8.8571, -12.8571],
[   17.2,    11.0,    15.6,      2.8,     2.2,    14.2,    15.6,    106.8,     5.2,      6.4],
[15.8333,  4.8333, -5.8333,      5.5, -9.6667,  7.6667,  5.6667,      9.5, 81.1667,     -3.0]])

B =


Matrix([
[ 310.2275],
[-193.5166],
[  238.112],
[-165.0229],
[ 214.4215],
[      0.0],
[      0.0],
[  28.8543],
[  291.732],
[-256.5717]])

A|B=


...

Khử


Matrix([
[129.553497102452,     0,                0,                0,                0,  4.74682622938878,                0,                0,                0,  21.9962995337415,  267.657048194028],
[               0, 156.0,                0,                0,                0, -3.36352506975332,                0,                0,                0,  24.2616420133449, -170.818645358579],
[               0,     0, 86.9305453885175,                0,                0,  11.8834889473867,                0,                0,                0,   12.447700783372,  266.552958203372],
[               0,     0,                0, 139.666697435897,                0, -1.49006361399572,                0,                0,                0,  1.47154060661322, -295.962219642608],
[               0,     0,                0,                0, 113.436805407526, -8.61003203148849,                0,                0,                0, -12.4193437565295,  124.287148050578],
[               0,     0,      

Chuẩn hoá


Matrix([
[1.0,   0,   0,   0,   0,  0.0366398926741049,   0,   0,   0,   0.169785455628007,  2.06599631951551],
[  0, 1.0,   0,   0,   0, -0.0215610581394443,   0,   0,   0,   0.155523346239391, -1.09499131640115],
[  0,   0, 1.0,   0,   0,   0.136700959303499,   0,   0,   0,   0.143191334274273,  3.06627500163574],
[  0,   0,   0, 1.0,   0, -0.0106687108763319,   0,   0,   0,  0.0105360879409969, -2.11906077165207],
[  0,   0,   0,   0, 1.0, -0.0759015735726744,   0,   0,   0,  -0.109482488614806,  1.09565098914829],
[  0,   0,   0,   0,   0,                   0,   0,   0,   0,                   0,                 0],
[  0,   0,   0,   0,   0,                   0,   0,   0,   0,                   0,                 0],
[  0,   0,   0,   0,   0,  0.0727028448667473, 1.0,   0,   0,  -0.150827285367454, 0.620220624616231],
[  0,   0,   0,   0,   0,  0.0969475488782495,   0, 1.0,   0,  0.0233092469865654,  2.16687538112257],
[  0,   0,   0,   0,   0,  0.0736776485769245,   0,   0, 1.0, -0

Số phần tử tự do là 2: X[6, 10].
